# Preprocessing
## 1. Techniques
Below contains functions to perform a series of preprocessing techniques on a given dataset.

In [3]:
#!pip install imblearn
from collections import Counter
from scipy.io import arff
from sklearn.model_selection import train_test_split as ts
from sklearn.model_selection import KFold
from imblearn.under_sampling import InstanceHardnessThreshold
import numpy as np
import pandas as pd

def Normalize(data):
    i_size = len(data) #The initial size of the data
    #Remove missing values
    r_data = data.dropna()
    r_size = len(r_data) #The data size after removing missing values
    if r_size/i_size > 0.8: #To ensure no false data removal was performed
        data = r_data
    #Remove outliers
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    result = data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1)]
    return result

def K_fold(data,f=10):
    kf = KFold(n_splits=10)
    tt_splits = kf.split(data)
    return tt_splits

def IHT(data):   
    gauss_iht = InstanceHardnessThreshold()
    underX, underY = gauss_iht.fit_resample(data[0],data[1])
    return underX, underY

## 2. Additional functions
Below are additional functions included to help manipulate the dataset, improving its usefulness

In [4]:
def data_conversion(data):
    for i in range(len(data)):
        if data[i] == b'N':
            data[i] = 0
        else:
            data[i] = 1
    return data

## 3. Main
Below contains the main algorithm which performs all the techniques on a given dataset.
#### It returns the preprocessed dataset with the following format:
An array with each row containing:
[Training metrics, Test metrics, Training labels, test labels]

In [9]:
def preprocess(data, fs = []):
    # raw = arff.loadarff(data)
    # data = pd.DataFrame(raw[0])
    # #Normalization
    # data = Normalize(data)
    #Pandas DF to NumPy array
    SM = np.array(data.iloc[:,:-1]) #Software metrics
    if fs != []:
        SM = SM[:,fs]
    L = data_conversion(np.array(data.iloc[:,-1])).astype(int) #Labels
    #K-fold
    tt_splits = K_fold(SM)
    #IHT
    result = []
    for train, test in tt_splits:
        SM_under, L_under = IHT([SM[train],L[train]])
        result.append([SM_under, SM[test], L_under, L[test]])
    return result

#File
if __name__=='__main__':
    result = preprocess('KC4.arff.txt')
    # for i in range(4):
    #     for j in range(4):
    #         print(f'{i} {j}: {result[i][j].shape}')

0 0: (54, 40)
0 1: (8, 40)
0 2: (54,)
0 3: (8,)
1 0: (52, 40)
1 1: (8, 40)
1 2: (52,)
1 3: (8,)
2 0: (60, 40)
2 1: (8, 40)
2 2: (60,)
2 3: (8,)
3 0: (48, 40)
3 1: (8, 40)
3 2: (48,)
3 3: (8,)
